In [1]:
import os
import requests
import json
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
WEATHER_API_KEY = os.getenv("WEATHER_API_KEY")

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
def get_weather(city="Seoul"):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_KEY}&units=metric&lang=kr"
    try:
        res = requests.get(url)
        res.raise_for_status()
    except Exception as e:
        return f"{city}의 날씨 정보를 불러오는 중 오류 발생: {e}"

    data = res.json()
    desc = data['weather'][0]['description']
    temp = data['main']['temp']
    return f"{city}의 현재 날씨는 {desc}, 기온은 {temp}도입니다."

def explain_weather(weather_info):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "너는 짧고 친절하게 날씨를 설명해주는 도우미야."},
            {"role": "user", "content": f"{weather_info} 이라고 할 때 외출 시 주의할 점을 알려줘."}
        ],
        max_tokens=150
    )
    return response.choices[0].message.content

city = "Seoul"
weather = get_weather(city)
print("날씨 정보 : ", weather)

response = explain_weather(weather)
print("🤖 GPT의 조언:", response)

날씨 정보 :  Seoul의 현재 날씨는 맑음, 기온은 13.98도입니다.
🤖 GPT의 조언: 현재 서울이 맑고 기온이 14도에 가까운 만큼, 가벼운 외출에 적합한 날씨입니다. 그러나 아침이나 저녁에는 다소 쌀쌀할 수 있으니 가벼운 겉옷을 챙기는 것이 좋습니다. 햇볕이 강할 수 있으니 sunscreen을 바르거나 모자를 착용하는 것도 추천드립니다. 즐거운 외출 되세요!


In [ ]:
def get_weather(city: str):
    url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={WEATHER_API_KEY}&units=metric&lang=kr"
    res = requests.get(url)
    if res.status_code != 200:
        return f"{city}의 날씨 정보를 불러올 수 없습니다."
    data = res.json()
    desc = data["weather"][0]["description"]
    temp = data["main"]["temp"]
    return f"{city}의 현재 날씨는 {desc}, 기온은 {temp}도입니다."

function_definitions = [
    {
        "name": "get_weather",
        "description": "특정 도시의 현재 날씨를 가져옵니다.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "도시 이름 (예: Seoul, Tokyo, New York)"
                }
            },
            "required": ["city"]
        }
    }
]

messages = [{"role": "user", "content": "서울 날씨는 어때"}] ## 질문

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    functions=function_definitions,
    function_call="auto"
)

choice = response.choices[0]
if choice.finish_reason == "function_call":
    func_call = choice.message.function_call
    func_name = func_call.name
    func_args = json.loads(func_call.arguments)

    if func_name == "get_weather":
        result = get_weather(**func_args)

        final_response = client.chat.completions.create(
            model="gpt-4o",
            messages=messages + [
                choice.message,
                {"role": "function", "name": func_name, "content": result}
            ]
        )
        print(final_response.choices[0].message.content)

else:
    print(choice.message.content)

현재 서울은 대체로 구름이 많고 기온은 약 18.76도입니다.
